In [1]:
import os
import random
import numpy as np
import torch
import config

/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


graphnet: INFO     2023-03-04 11:55:01 - get_logger - Writing log to logs/graphnet_20230304-115501.log


In [2]:
def set_gpu_environ():
    """Sets CUDA_VISIBLE_DEVICES to those under minimal memory load.
    Meant to be used in notebooks only.
    """
    import os
    import subprocess
    query = subprocess.check_output(['nvidia-smi', '--query-gpu=memory.used', '--format=csv']).decode().split('\n')[1:-1]
    utilization = [int(x.replace(" MiB", "")) for x in query]
    free = [i for i in range(len(utilization)) if utilization[i] == min(utilization)]
    set_visible = ",".join([str(i) for i in free])
    os.environ["CUDA_VISIBLE_DEVICES"] = set_visible
    print(set_visible)
set_gpu_environ()

0


In [3]:
def seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def train(cfg):
    seed()
    custom_model = cfg.MODEL_NAME()
    if cfg.MODEL_WTS:
        print(f"Loading model weights from {cfg.MODEL_WTS}")
        custom_model.load_state_dict(torch.load(cfg.MODEL_WTS))
    opt = cfg.OPT(
        custom_model.parameters(), lr=cfg.LR, weight_decay=cfg.WD
    )
    loss_func = cfg.LOSS_FUNC()
    len_trn_dl = (cfg.N_FILES * 200000)//cfg.BATCH_SIZE
    warmup_steps = int(len_trn_dl * cfg.WARM_UP_PCT * cfg.EPOCHS)
    total_steps = int(len_trn_dl * cfg.EPOCHS)

    print(f"Total steps: {total_steps}, Warmup steps: {warmup_steps}")

    scheduler = cfg.SCHEDULER(
        opt,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps,
    )

    cfg.FIT_FUNC(
        epochs=cfg.EPOCHS,
        model=custom_model,
        loss_fn=loss_func,
        opt=opt,
        metric=cfg.METRIC,
        config = cfg,
        folder=cfg.FOLDER/cfg.EXP_NAME,
        exp_name=f"{cfg.EXP_NAME}",
        device=cfg.DEVICE,
        sched=scheduler,
    )


In [4]:
def main(config_name):
    configs = eval(f"config.{config_name}")
    print(f"Training with config: {configs.__dict__}")
    os.makedirs(configs.FOLDER/configs.EXP_NAME)
    train(configs)

In [ ]:
main('BASELINE_graph_V14_FT')

Training with config: {'__module__': 'config', 'MODEL_WTS': '/opt/slh/icecube/RESULTS/EXP_40/EXP_40_0.pth', 'EXP_NAME': 'EXP_40_FT', 'FILTER': False, 'LOSS_FUNC': <class 'icecube.modelsgraph.gVonMisesFisher3DLoss'>, 'NUM_WORKERS': 22, 'MODEL_NAME': <class 'icecube.modelsgraph.GraphxTransformerV3'>, 'FIT_FUNC': <function gfit_shuflle_f32 at 0x7f1edc280200>, 'METRIC': <function gget_score_vector at 0x7f1edc280560>, 'TRN_DATASET': <class 'icecube.graphdataset.GraphDasetV0'>, 'VAL_DATASET': <class 'icecube.graphdataset.GraphDasetV0'>, 'TRN_BATCH_RANGE': [[101, 151], [1, 51], [151, 201], [251, 301], [201, 251], [301, 351], [401, 451], [351, 401], [451, 501], [551, 601], [601, 651], [501, 551], [51, 101]], 'EPOCHS': 13, 'DEVICE': 'cuda:0', 'BATCH_SIZE': 512, 'VAL_BATCH_RANGE': (655, 660), 'LR': 0.0005, 'WD': 0.05, 'N_FILES': 50, 'WARM_UP_PCT': 0.01, '__doc__': None}
Loading model weights from /opt/slh/icecube/RESULTS/EXP_40/EXP_40_0.pth
Total steps: 253903, Warmup steps: 2539


wandb: Currently logged in as: drhb (kaggle-hi). Use `wandb login --relogin` to force relogin


epoch,train_loss,valid_loss,val_metric


trn_range: [101, 151]
Better model found at epoch 0 with value: 1.0588619709014893.
   epoch  train_loss  valid_loss    metric
0      0    1.517539    1.549849  1.058862
trn_range: [1, 51]


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Better model found at epoch 3 with value: 1.0445926189422607.
   epoch  train_loss  valid_loss     metric
0      3    1.581381    1.588248  1.0445926
trn_range: [201, 251]


In [ ]:
#6 min
NUM_WORKERS = 22
BATCH_SIZE = 512

In [ ]:
k

In [ ]:
f

In [ ]:
lk

In [ ]:
l